In [3]:
# gradient boosting
import os
import pandas as pd
import numpy as np
import catboost

In [166]:
NUMBER_NODES = 7

def load_data():
    train_df = pd.read_csv(os.path.join('..', 'datasets', f'dataset_{NUMBER_NODES}_train.csv'))
    val_df = pd.read_csv(os.path.join('..', 'datasets', f'dataset_{NUMBER_NODES}_val.csv'))
    test_df = pd.read_csv(os.path.join('..', 'datasets', f'dataset_{NUMBER_NODES}_test.csv'))

    featuresNumber = (NUMBER_NODES * NUMBER_NODES - NUMBER_NODES) // 2 
    def get_tuple_tensor_dataset(row):
        X = row[0 : featuresNumber].astype('int32')
        Y = row[featuresNumber + 1: ].astype('int32') # Inclui a banda otima na posicao 0
        return X, Y

    train_dataset = list(map(get_tuple_tensor_dataset, train_df.to_numpy()))
    val_dataset = list(map(get_tuple_tensor_dataset, val_df.to_numpy()))
    test_dataset = list(map(get_tuple_tensor_dataset, test_df.to_numpy()))

    X = []
    Y = []
    for x, y in train_dataset:
        X.append(x)
        Y.append(y)
    x_train = np.array(X)
    y_train = np.array(Y)

    X = []
    Y = []
    for x, y in test_dataset:
        X.append(x)
        Y.append(y)
    x_test = np.array(X)
    y_test = np.array(Y)

    X = []
    Y = []
    for x, y in val_dataset:
        X.append(x)
        Y.append(y)
    x_val = np.array(X)
    y_val = np.array(Y)

    x_train = np.concatenate((x_train, x_val))
    y_train = np.concatenate((y_train, y_val))


    return x_train, y_train, x_test, y_test

In [168]:
X, y, x_t, y_t = load_data()

In [169]:
cat_features = list(range(0, X.shape[1]))
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


In [182]:
from catboost import CatBoostRegressor

# cat_features is the categorical features indices

model = CatBoostRegressor(objective='MultiRMSE', verbose=100)
model.fit(X, y, cat_features=cat_features)

0:	learn: 4.9685372	total: 9.62ms	remaining: 9.61s
100:	learn: 4.2595071	total: 1.36s	remaining: 12.1s
200:	learn: 3.9160863	total: 2.67s	remaining: 10.6s
300:	learn: 3.6589244	total: 4.02s	remaining: 9.34s
400:	learn: 3.4594657	total: 5.37s	remaining: 8.02s
500:	learn: 3.2926053	total: 6.74s	remaining: 6.71s
600:	learn: 3.1553447	total: 8.09s	remaining: 5.37s
700:	learn: 3.0330676	total: 9.43s	remaining: 4.02s
800:	learn: 2.9279096	total: 10.7s	remaining: 2.67s
900:	learn: 2.8332496	total: 12.1s	remaining: 1.33s
999:	learn: 2.7492686	total: 13.4s	remaining: 0us


In [183]:
a = model.predict(x_t)
print(a[1])
print(np.round(a[1]))

[1.45526307 1.30165131 4.07261749 3.93628902 5.25165878 2.03353679
 2.94898353]
[1. 1. 4. 4. 5. 2. 3.]


In [184]:
def count_repeats(output):
    x = [x for x in output if x > 6]
    greaterSix = len(x)
    true_shape = NUMBER_NODES
    counts = np.unique(np.round(output))
    repeated = true_shape - counts.shape[0]
    return repeated, greaterSix

count = 0
greaterSix = 0
for i in a:
    c, g = count_repeats(i)
    count += c
    greaterSix += g
print(count)
print(greaterSix)

185
1


In [185]:
print(f"Com repetição - {count}") 
# 185, 1 com mais dados
# 144, 24 com mais dados e 10k

Com repetição - 185
